In [185]:
from langchain_community.document_loaders import FireCrawlLoader
from langchain_community.document_loaders import Docx2txtLoader
import threading

from langchain_core.pydantic_v1 import BaseModel, Field, validator, root_validator
# Load HTML
from langchain_community.document_loaders import FireCrawlLoader
from langchain_groq import ChatGroq
from typing import List
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
import os
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import ChatOpenAI
from typing import List

os.environ["GROQ_API_KEY"] = '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'
open_ai_key = 'sk-None-YYN2MievH2m1islKXEIkT3BlbkFJ9vmzUu8bHOOXgK9498jA'
os.environ["OPENAI_API_KEY"] = open_ai_key
llm=ChatOpenAI(temperature=0.0, model_name="gpt-4o") 

In [186]:
def get_markdown_content(page_content):
    llm = ChatGroq(
            model="llama-3.1-70b-versatile",
            temperature=0,
            max_tokens=None,
            timeout=None,
            max_retries=3,
            # other params...
        )
    
    prompt = PromptTemplate(
            template=" You will be given a page which contain link and other html stuff. You need to clean it and return only text. No need for images link or anything like that.\n \n{query}\n",
            input_variables=["query"],
        )
    chain = prompt | llm  

    result = chain.invoke({"query": page_content})
    return result

In [187]:
def get_html(url,mode='scrape',ignore_pages=[],max_pages=4):
    if mode == 'scrape':
        loader = FireCrawlLoader(
        api_key="fc-d22f7f0d1f7c46949c24b237959797f6", url=url, mode=mode
            )
    else:
        crawl_params = {
                'crawlerOptions': {
                    'excludes': [
                        'blog/*', 'login/*', 'account/*', 'user/*', 'profile/*',
                        'admin/*', 'dashboard/*', 'search/*', 'filter/*',
                        'checkout/*', 'payment/*', 'cart/*', 'css/*', 'js/*',
                        'images/*', 'assets/*', 'temp/*', 'under-construction/*',
                        'api/*', '404', '500', 'downloads/*', 'files/*', 'pdfs/*',
                        'archive/*', 'old/*', 'version/*', 'forum/*', 'comments/*',
                        'reviews/*', 'external/*', 'outbound/*', 'product/*', 'shop/*',
                        'category/*', 'promo/*', 'deals/*', 'offers/*', 'help/*',
                        'support/*', 'news/*', 'press/*', 'events/*',
                        'calendar/*', 'subscribe/*', 'signup/*', 'test/*', 'staging/*',
                        'wp-admin/*', 'backend/*', 'admin-panel/*', 'management/*'
                    ],
                    'includes': [], # leave empty for all pages
                    'limit': max_pages,
                }
            }

        loader = FireCrawlLoader(
        api_key="fc-d22f7f0d1f7c46949c24b237959797f6", url=url, mode=mode,
        params=crawl_params
            ) 
    docs = loader.load()
    # print(docs)
    return docs

In [189]:
html_content = get_html("https://aman.ai/cs229/gda/")

In [190]:
def remove_all_repetitions(text):
    words = text.split()
    seen = set()
    result = []

    for word in words:
        if word not in seen:
            seen.add(word)
            result.append(word)

    return ' '.join(result)


In [191]:
import re


def extract_plain_text_from_markdown(markdown_content):
    # Remove image tags (Markdown and HTML)
    markdown_content = re.sub(r'!\[.*?\]\(.*?\)', '', markdown_content)
    markdown_content = re.sub(r'<img.*?>', '', markdown_content)
    
    # Remove links (Markdown and plain URLs)
    markdown_content = re.sub(r'\[.*?\]\(.*?\)', '', markdown_content)
    markdown_content = re.sub(r'\(https?://.*?\)', '', markdown_content)
    markdown_content = re.sub(r'https?://\S+|www\.\S+', '', markdown_content)
    
    # Remove any remaining inline links
    markdown_content = re.sub(r'\[.*?\]', '', markdown_content)
    
    # Remove HTML tags (including SVG)
    markdown_content = re.sub(r'<.*?>', '', markdown_content)
    
    # Remove all symbols except ., "", '', ?, and ,
    markdown_content = re.sub(r"[^a-zA-Z0-9\s.,\"'?']", '', markdown_content)
    
    # Remove any extra whitespace and newlines
    markdown_content = re.sub(r'\s+', ' ', markdown_content).strip()
    
    return markdown_content

In [192]:
cleaned_content = extract_plain_text_from_markdown(html_content[0].page_content)

In [193]:
cleaned_content = remove_all_repetitions(cleaned_content)

In [194]:
cleaned_content

'Back to Top CS229 Gaussian Discriminant Analysis Generative Learning Algorithms Weve mainly looked at learning algorithms that model Pyx, the conditional distribution of y given x. For instance, logistic regression modeled Pyx as hxgTx where g is sigmoid function. In this section, well talk about a different type algorithm. Lets use binary classification problem motivation behind our discussion. Consider in which we want learn distinguish between malignant y1 and benign y0 tumors. Given training set, an algorithm like , or initially starts with randomly initialized parameters. Over course learning, performs gradient descent straight line hyperplane decision boundary evolves until you obtain separates positivenegative examples case, Then, classify new sample either benign, it checks on side falls in, makes its prediction accordingly. Theres class arent trying maximize likelihood looking both classes searching for separation boundary. Instead, these look one time. First, tumors, can fea

In [195]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [196]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=8000,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)

In [197]:
texts = text_splitter.create_documents([cleaned_content])

In [198]:
len(texts)

2

In [199]:
from typing import Optional

In [200]:
from typing import List, Optional
from pydantic import BaseModel, Field, root_validator
from pydantic import BaseModel, Field, root_validator, ValidationError
from langchain.chains import LLMChain

In [201]:

# Define SentenceParser model
class SentenceParser(BaseModel):
    originalContent: str = Field(..., description="Original Sentence with no correction")
    correctedSentence: str = Field(..., description="Corrected Sentence in English UK. Enclosed corrections in <b></b>.")

    @root_validator(pre=True)
    def fill_missing_fields(cls, values):
        values['correctedSentence'] = values.get('correctedSentence', '')
        return values

# Define ResponseModel to include sentences
class ResponseModel(BaseModel):
    sentences: List[SentenceParser]

# Function to get the markdown content
def get_markdown_content(page_content):
    llm = ChatGroq(
            model="llama-3.1-70b-versatile",
            temperature=0,
            max_tokens=None,
            timeout=None,
            max_retries=3,
            # other params...
        )
    

    parser = PydanticOutputParser(pydantic_object=ResponseModel)

    prompt = PromptTemplate(
        template="""
        You are a smart assistant.
        You will be given a page-long document.
        You will return a list of sentences in the page.
        If there is any HTML content or non-textual content, you will remove it.
        First, you will clean the text by removing the images and links, and extract plain text.
        You will not expand any point or enhance anything. 
        If sentence contain the social media or web development related things like cookies etc you will remove it.
        Just return what is in the original text.
        Provide the English UK version of sentences, performing both spelling and grammar checks.
        Return the result as a JSON object with a single key "sentences" which contains a list of objects.
        Each object should have keys "originalContent" and "correctedSentence".

        \n\n{query}\n
        """,
        input_variables=["query"]
    )

    chain = LLMChain(llm=llm, prompt=prompt, output_parser=parser)

    result = chain({"query": page_content})

    if isinstance(result,dict):
        result = result['text']
        return result
    return result

/var/folders/gv/v0q4rb4n3p9_ydb839lgywph0000gq/T/ipykernel_8980/3567739227.py:6: PydanticDeprecatedSince20: Pydantic V1 style `@root_validator` validators are deprecated. You should migrate to Pydantic V2 style `@model_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  @root_validator(pre=True)


In [202]:
text_random = texts[0]

In [203]:
text_random

Document(page_content='Back to Top CS229 Gaussian Discriminant Analysis Generative Learning Algorithms Weve mainly looked at learning algorithms that model Pyx, the conditional distribution of y given x. For instance, logistic regression modeled Pyx as hxgTx where g is sigmoid function. In this section, well talk about a different type algorithm. Lets use binary classification problem motivation behind our discussion. Consider in which we want learn distinguish between malignant y1 and benign y0 tumors. Given training set, an algorithm like , or initially starts with randomly initialized parameters. Over course learning, performs gradient descent straight line hyperplane decision boundary evolves until you obtain separates positivenegative examples case, Then, classify new sample either benign, it checks on side falls in, makes its prediction accordingly. Theres class arent trying maximize likelihood looking both classes searching for separation boundary. Instead, these look one time. 

In [204]:
mkd_contents = get_markdown_content(text_random)

In [205]:
out_sentences = mkd_contents.dict().get('sentences')

/var/folders/gv/v0q4rb4n3p9_ydb839lgywph0000gq/T/ipykernel_8980/985300905.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  out_sentences = mkd_contents.dict().get('sentences')


In [206]:
out_sentences

[{'originalContent': 'Back to Top CS229 Gaussian Discriminant Analysis Generative Learning Algorithms Weve mainly looked at learning algorithms that model Pyx, the conditional distribution of y given x.',
  'correctedSentence': "Back to Top CS229 Gaussian Discriminant Analysis Generative Learning Algorithms We've mainly looked at learning algorithms that model Pyx, the conditional distribution of y given x."},
 {'originalContent': 'For instance, logistic regression modeled Pyx as hxgTx where g is sigmoid function.',
  'correctedSentence': 'For instance, logistic regression modelled Pyx as hxgTx where g is sigmoid function.'},
 {'originalContent': 'In this section, well talk about a different type algorithm.',
  'correctedSentence': "In this section, we'll talk about a different type of algorithm."},
 {'originalContent': 'Lets use binary classification problem motivation behind our discussion.',
  'correctedSentence': "Let's use binary classification problem motivation behind our discus

In [207]:
from difflib import ndiff

In [208]:
data = out_sentences

In [209]:
data

[{'originalContent': 'Back to Top CS229 Gaussian Discriminant Analysis Generative Learning Algorithms Weve mainly looked at learning algorithms that model Pyx, the conditional distribution of y given x.',
  'correctedSentence': "Back to Top CS229 Gaussian Discriminant Analysis Generative Learning Algorithms We've mainly looked at learning algorithms that model Pyx, the conditional distribution of y given x."},
 {'originalContent': 'For instance, logistic regression modeled Pyx as hxgTx where g is sigmoid function.',
  'correctedSentence': 'For instance, logistic regression modelled Pyx as hxgTx where g is sigmoid function.'},
 {'originalContent': 'In this section, well talk about a different type algorithm.',
  'correctedSentence': "In this section, we'll talk about a different type of algorithm."},
 {'originalContent': 'Lets use binary classification problem motivation behind our discussion.',
  'correctedSentence': "Let's use binary classification problem motivation behind our discus

In [210]:
from difflib import ndiff

def highlight_changes(original, corrected):
    diff = list(ndiff(original.split(), corrected.split()))
    result = []
    in_change = False

    for d in diff:
        if d.startswith('- '):
            continue
        elif d.startswith('+ '):
            if not in_change:
                result.append('<b>')
                in_change = True
            result.append(d[2:])
        elif d.startswith('  '):
            if in_change:
                result.append('</b>')
                in_change = False
            result.append(d[2:])
    
    if in_change:
        result.append('</b>')

    return ' '.join(result)

# Test
original_text = "This is the original text"
corrected_text = "This is the corrected text"
print(highlight_changes(original_text, corrected_text))

This is the <b> corrected </b> text


In [211]:
highlighted_sentences = [highlight_changes(pair['originalContent'], pair['correctedSentence']) for pair in data]
highlighted_sentences

["Back to Top CS229 Gaussian Discriminant Analysis Generative Learning Algorithms <b> We've </b> mainly looked at learning algorithms that model Pyx, the conditional distribution of y given x.",
 'For instance, logistic regression <b> modelled </b> Pyx as hxgTx where g is sigmoid function.',
 "In this section, <b> we'll </b> talk about a different type <b> of </b> algorithm.",
 "<b> Let's </b> use binary classification problem motivation behind our discussion.",
 'Consider in which we want <b> to </b> learn <b> to </b> distinguish between malignant y1 and benign y0 <b> tumours. </b>',
 'Given <b> a </b> training set, an algorithm like , or initially starts with randomly <b> initialised </b> parameters.',
 'Over <b> the </b> course <b> of </b> learning, <b> it </b> performs gradient descent <b> on a </b> straight line hyperplane decision <b> boundary, which </b> evolves until <b> it </b> separates <b> positive and negative examples. </b> Then, <b> it classifies a </b> new sample <b> as 

In [212]:
import pandas as pd

In [213]:
df = pd.DataFrame(out_sentences)

In [214]:
df

,originalContent,correctedSentence
0,Back to Top CS229 Gaussian Discriminant Analys...,Back to Top CS229 Gaussian Discriminant Analys...
1,"For instance, logistic regression modeled Pyx ...","For instance, logistic regression modelled Pyx..."
2,"In this section, well talk about a different t...","In this section, we'll talk about a different ..."
3,Lets use binary classification problem motivat...,Let's use binary classification problem motiva...
4,Consider in which we want learn distinguish be...,Consider in which we want to learn to distingu...
...,...,...
125,"3. converse, implication does Gaussian.","The converse is also true, and the implication..."
126,"3 2. stronger prove 2, noted property GDA.","3 implies 2, and this is a stronger result. We..."
127,"correct, better regression.","This is correct, and better than regression."
128,"Specifically, indeed asymptotically efficient.","Specifically, it is indeed asymptotically effi..."


In [215]:
df['changes'] = highlighted_sentences

In [216]:
filtered_df = df[df['changes'].str.contains('<b>')]

In [217]:
def normalize_and_filter(df, column_name):
    # Create a temporary column in lowercase for normalization and matching
    df['temp_lower'] = df[column_name].str.lower()
    
    # Pattern to match social media names, specific extensions, and keywords like 'cookies'
    pattern = r'\b(company|facebook|twitter|instagram|linkedin|youtube|\.com|\.ad|\.gov|\.uk|\.php|\.asp|cookies|cookies|statistical|tracking|ads|script|event|policy|Cloudflare|cookie|cfduid|llc|party)\b'
    
    # Filter rows where the temporary lowercase column does not contain any of the specified patterns
    filtered_df = df[~df['temp_lower'].str.contains(pattern, flags=re.IGNORECASE, regex=True)]
    
    # Drop the temporary column
    filtered_df.drop('temp_lower', axis=1, inplace=True)
    
    return filtered_df

In [218]:
filtered_df = normalize_and_filter(df, 'changes')

/var/folders/gv/v0q4rb4n3p9_ydb839lgywph0000gq/T/ipykernel_8980/3883782903.py:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered_df = df[~df['temp_lower'].str.contains(pattern, flags=re.IGNORECASE, regex=True)]


In [219]:
filtered_df.iloc[0]['changes']


"Back to Top CS229 Gaussian Discriminant Analysis Generative Learning Algorithms <b> We've </b> mainly looked at learning algorithms that model Pyx, the conditional distribution of y given x."

In [220]:
filtered_df.iloc[0]['originalContent']

'Back to Top CS229 Gaussian Discriminant Analysis Generative Learning Algorithms Weve mainly looked at learning algorithms that model Pyx, the conditional distribution of y given x.'

In [221]:
df.to_json()

'{"originalContent":{"0":"Back to Top CS229 Gaussian Discriminant Analysis Generative Learning Algorithms Weve mainly looked at learning algorithms that model Pyx, the conditional distribution of y given x.","1":"For instance, logistic regression modeled Pyx as hxgTx where g is sigmoid function.","2":"In this section, well talk about a different type algorithm.","3":"Lets use binary classification problem motivation behind our discussion.","4":"Consider in which we want learn distinguish between malignant y1 and benign y0 tumors.","5":"Given training set, an algorithm like , or initially starts with randomly initialized parameters.","6":"Over course learning, performs gradient descent straight line hyperplane decision boundary evolves until you obtain separates positivenegative examples case, Then, classify new sample either benign, it checks on side falls in, makes its prediction accordingly.","7":"Theres class arent trying maximize likelihood looking both classes searching for separa

In [222]:
df_dict = df.to_dict(orient='records')

In [223]:
data = df_dict

In [224]:
filtered_df = df[df['changes'].str.contains('<b>')]

In [226]:
filtered_df.to_csv('a_data.csv', index=False)

In [227]:
sentences = list(filtered_df['originalContent'])

In [228]:
caution_word_list = ["Best", "Specialist", "Specialised", "Finest", "Most experienced",
                                "Superior", "Principle", "Expert", "Amazing", "Speciality",
                                "leader", "leaders", "service", "implantologist"
            ]

In [229]:


# Function to add <b></b> around caution words
def highlight_caution_words(sentence):
    pattern = re.compile(r'\b(' + '|'.join(re.escape(word) for word in caution_word_list) + r')\b', re.IGNORECASE)
    highlighted_sentence = pattern.sub(r'<b>\1</b>', sentence)
    if highlighted_sentence != sentence:
        return highlighted_sentence
    return None

# Apply the function to all sentences and filter out None results
highlighted_sentences = [highlight_caution_words(sentence) for sentence in sentences if highlight_caution_words(sentence)]

# Print the results
for sentence in highlighted_sentences:
    print(sentence)

discuss <b>superior</b> viceversa.
